<a href="https://colab.research.google.com/github/Mushtatoes/proj/blob/master/Pokemon_Type_1_to_Highest_Base_Stat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pokemon - Base Stats and Types
**Description:** Is there a correlation between a pokemon's primary type and its highest stat? 


**Goal:** Find a pattern in the minds of pokemon designer and learn more about pandas and matplotlib.
<center>
<img src='https://cdn.clipart.email/cb04f2d0395acc25e2abe9da2b289a5a_pikachu-transparent-background-png-mart_1254-1254.png' height='300'> 
</center>

>For example, rock pokemon might generally have defense as their highest stat.
Maybe Psychic pokemon might have generally high special attack. 
Are there any outlier pokemon that may be a bit different from others of the same type?




In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
import re

## 1. Obtain data for all existing Pokemon:


>- No mega-evolutions, alternate forms, etc.
- Pulling from PokeAPI
- Manually parse and add data into Dataframe without using read_json
- At time of writing, there are currently 807 unique Pokemon in the API (Up to Gen 7)




In [0]:
pokemon_json = requests.get('https://pokeapi.co/api/v2/pokemon?&limit=5000').json()
# print(pokemon_json)

#### 1a. Filter out non-standard Pokemon by id:
>- Create a list of urls that can be used to pull data on each Pokemon.
- PokeAPI uses id <= 10000 to identify standard Pokemon and their traits.
- Use regular expressions to isolate the id from the url and compare.

In [0]:
name_url_raw = pokemon_json['results']

# Filter out non-standard Pokemon using regex on url:
def pokemon_filter(pokemon_object):
  pokemon_search = re.search('^https://pokeapi\.co/api/v2/pokemon/(\d+)/$', pokemon_object['url'])
  if not pokemon_search:
    return False
  return int(pokemon_search.group(1)) <= 10000

name_url = list(filter(pokemon_filter, name_url_raw))

pokemon_urls = [x['url'] for x in name_url]


#### 1b. Map the required data:
>- Each Pokemon's base stats can only be found on their own page, which means a request for each Pokemon.
- Storing into a flat file or SQL table would be preferable if the data is  required for reuse in the future.

In [0]:
# Return each pokemon's name, type1, and base stats:
def pokemon_map(url):
  pokemon = requests.get(url).json()
  data = {'name': pokemon['name']
          ,'type1': ''.join([t['type']['name'] for t in pokemon['types'] if t['slot'] == 1])
          ,'stats': {s['stat']['name']: s['base_stat']for s in pokemon['stats']}}
  return data
  
pokemon_data = list(map(pokemon_map, pokemon_urls))

# print(pokemon_data)

#### 1c. Push the data into a pandas DataFrame:
> - Names and types can be extracted into separate lists.
- Each pokemon has 6 base stats:
  - hp
  - attack
  - special-attack
  - defense
  - special-defense
  - speed
- These will have to each be placed into separate lists to push into the DataFrame.

In [5]:
# Format data into dictionaries for DataFrame:
# {Header: [data]}

def pivot_data(category, subcategory = None):
  if not subcategory:
    return [pokemon[category] for pokemon in pokemon_data]
  else:
    return [pokemon[category][subcategory] for pokemon in pokemon_data]

pokemon_name = pivot_data('name')
pokemon_type1 = pivot_data('type1')
pokemon_spd = pivot_data('stats', 'speed')
pokemon_spdef = pivot_data('stats', 'special-defense')
pokemon_spatk = pivot_data('stats', 'special-attack')
pokemon_def = pivot_data('stats', 'defense')
pokemon_atk = pivot_data('stats', 'attack')
pokemon_hp = pivot_data('stats', 'hp')

pokemon_df = pd.DataFrame({'name': pokemon_name
                           ,'type1': pokemon_type1
                           ,'spd': pokemon_spd
                           ,'spdef': pokemon_spdef
                           ,'spatk': pokemon_atk
                           ,'def': pokemon_def
                           ,'atk': pokemon_atk
                           ,'hp': pokemon_hp})

# Display top rows from DataFrame for preview:

pokemon_df.head()

,name,type1,spd,spdef,spatk,def,atk,hp
0,bulbasaur,grass,45,65,49,49,49,45
1,ivysaur,grass,60,80,62,63,62,60
2,venusaur,grass,80,100,82,83,82,80
3,charmander,fire,65,50,52,43,52,39
4,charmeleon,fire,80,65,64,58,64,58


## 2. Analysis on highest base stat per type:


>- Now that the data is in, any operations can be performed.
- idxmax can return the column name to get the highest stat for each row.


In [6]:
pokemon_df['hi_stat'] = pokemon_df[['spd','spdef','spatk','def','atk','hp']].idxmax(axis=1)

pokemon_df.head()

,name,type1,spd,spdef,spatk,def,atk,hp,hi_stat
0,bulbasaur,grass,45,65,49,49,49,45,spdef
1,ivysaur,grass,60,80,62,63,62,60,spdef
2,venusaur,grass,80,100,82,83,82,80,spdef
3,charmander,fire,65,50,52,43,52,39,spd
4,charmeleon,fire,80,65,64,58,64,58,spd


#### 2a. Aggregate data:
> - Count the number of highest base stats per type and display.
- Move each type into their individual DataFrames.


In [7]:
df_types = pokemon_df.type1.unique()

grass_df = pokemon_df.loc[pokemon_df['type1'] == 'grass']

grass_df


,name,type1,spd,spdef,spatk,def,atk,hp,hi_stat
0,bulbasaur,grass,45,65,49,49,49,45,spdef
1,ivysaur,grass,60,80,62,63,62,60,spdef
2,venusaur,grass,80,100,82,83,82,80,spdef
42,oddish,grass,30,65,50,55,50,45,spdef
43,gloom,grass,40,75,65,70,65,60,spdef
...,...,...,...,...,...,...,...,...,...
760,bounsweet,grass,32,38,30,38,30,42,hp
761,steenee,grass,62,48,40,48,40,52,spd
762,tsareena,grass,72,98,120,98,120,72,spatk
786,tapu-bulu,grass,75,95,130,115,130,70,spatk
